In [10]:
from __future__ import division
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.layers import dropout

In [11]:
#load data
daysActivities = pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/r相关函数/normaldnn/save/daysActivities.csv',
                      header=0,encoding='gbk' )

In [13]:
user_day = daysActivities.ix[:,:2]
daysActivities = np.array(daysActivities.ix[:,2:],dtype=np.float32)
rowNum = len(daysActivities)
colNum = len(daysActivities[0])
trainNums =int(rowNum*0.9)
trainData = daysActivities[:trainNums]
testData = daysActivities[trainNums:]

missData = np.array([np.zeros(colNum)],dtype=np.float32)

In [14]:
n_inputs = colNum
n_hidden1 = n_inputs//3
n_hidden2 = n_hidden1//3
n_hidden3 = n_hidden2//3
n_hidden4 = n_hidden2
n_hidden5 = n_hidden1
n_outpus = n_inputs

learning_rate=0.0001

In [15]:
activation = tf.nn.elu
initializer = tf.contrib.layers.variance_scaling_initializer()
is_training = tf.placeholder_with_default(False, shape=(), name='is_training')
keep_prob = 0.7

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
X_drop = dropout(X, keep_prob, is_training=is_training)

weights1_init = initializer([n_inputs, n_hidden1])
weights2_init = initializer([n_hidden1, n_hidden2])
weights3_init = initializer([n_hidden2, n_hidden3])

weights1 = tf.Variable(weights1_init, dtype=tf.float32, name="weights1")
weights2 = tf.Variable(weights2_init, dtype=tf.float32, name="weights2")
weights3 = tf.Variable(weights3_init, dtype=tf.float32, name="weights3")

weights4 = tf.transpose(weights3, name="weights4") # tied weights
weights5 = tf.transpose(weights2, name="weights5") # tied weights
weights6 = tf.transpose(weights1, name="weights6") # tied weights

biases1 = tf.Variable(tf.zeros(n_hidden1), name="biases1")
biases2 = tf.Variable(tf.zeros(n_hidden2), name="biases2")
biases3 = tf.Variable(tf.zeros(n_hidden3), name="biases3")
biases4 = tf.Variable(tf.zeros(n_hidden4), name="biases4")
biases5 = tf.Variable(tf.zeros(n_hidden5), name="biases5")
biases6 = tf.Variable(tf.zeros(n_outpus),  name="biases6")

hidden1 = activation(tf.matmul(X_drop,  weights1) + biases1)
hidden2 = activation(tf.matmul(hidden1, weights2) + biases2)
hidden3 = activation(tf.matmul(hidden2, weights3) + biases3)
hidden4 = activation(tf.matmul(hidden3, weights4) + biases4)
hidden5 = activation(tf.matmul(hidden4, weights5) + biases5)
outputs = tf.matmul(hidden5, weights6) + biases6

codings = hidden3

loss = tf.reduce_mean(tf.square(outputs - X)) 


optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()

In [17]:
n_epochs = 30
batch_size = 1000
batch_num = len(daysActivities) // batch_size+1
with tf.Session() as sess:
    init.run()
    train_loss_eval = loss.eval(feed_dict={X: trainData})
    test_loss_eval  = loss.eval(feed_dict={X: testData})
    print(train_loss_eval,test_loss_eval)
    for epoch in range(n_epochs):  
        for batch in range(batch_num):
            X_batch, y_batch = trainData[batch*batch_size:(batch+1)*batch_size],1
            sess.run(training_op, feed_dict={X: X_batch,is_training:True})
        if epoch%10==0:
            train_loss_eval = loss.eval(feed_dict={X: trainData})
            test_loss_eval  = loss.eval(feed_dict={X: testData})
            print(train_loss_eval,test_loss_eval)
    codings_evl  = codings.eval(feed_dict={X: daysActivities})
    miss_codings = codings.eval(feed_dict={X:missData})

0.0278938 0.0305977
0.0118577 0.0132249
0.00433203 0.00476453
0.00360745 0.00393782


In [18]:
codings_evl = pd.DataFrame(codings_evl)
codings_evl[['user_id','day']]=user_day
default_codings = pd.DataFrame(miss_codings)

In [21]:
codings_evl.to_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/r相关函数/normaldnn/save/daysActivities_prop_coding.csv')
default_codings.to_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/r相关函数/normaldnn/save/daysActivities_prop_default_coding.csv')